In [2]:
import os

02_feature_engineering_pipeline

Objective:
    1. Reading the file from the loacation
    2. Doing data preprcessing like extracting relevant features, maping values.
    3. Let's do 1 and 2

feature_engineering: \
  root_dir: artifacts/feature_engineering  \
  training_data_path: artifacts/feature_engineering  \
  training_data_file: artifacts/feature_engineering/tweets.cs


In [1]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    training_data_path: Path
    training_data_file: Path
    training_cleansed_data: Path
    

In [4]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT\\research'

In [3]:
import os
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT'

In [10]:
from pathlib import Path

CONFIG_FILE_PATH = Path("c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT/config/config.yaml")
PARAMS_FILE_PATH = Path("c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT/params.yaml")

In [11]:
from src.textClassifier.constants import *
from src.textClassifier.utils.commons import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.feature_engineering

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            training_data_path=config.training_data_path,
            training_data_file=config.training_data_file,
            training_cleansed_data=config.training_cleansed_data
        )

        return data_preprocessing_config

In [37]:
# from textClassifier import logger
# import zipfile
# import os
# from textClassifier.utils.commons import get_size
# import gdown
# import spacy
# from nltk.corpus import stopwords
# import string



# class DataPreprocessing:
#     def __init__(self, config: DataPreprocessingConfig):
#         self.config = config

#     def preprocess_text(self):
#         nlp = spacy.load('en_core_web_sm')
#         stop_words = set(stopwords.words('english'))
#         text = text.lower() # Convert to lowercase

#         # Remove punctuation
#         text = text.translate(str.maketrans('', '', string.punctuation))

#         # Tokenize and lemmatize
#         doc = nlp(text)
#         tokens = [token.lemma_ for token in doc if token.text not in stop_words]
#         return ' '.join(tokens)

#     def pass_data_to_text_process(self):
#         # self.config.training_data_file
#         data['cleaned_text'] = data['text'].apply(preprocess_text)
#         return data


In [38]:
# dp = DataPreprocessing(config_manager)

C:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\artifacts\data_ingestion\Tweets.csv

In [39]:
# dp.preprocess_text('hi dude Happy was how are you doing')

'hi dude happy'

In [1]:
from textClassifier import logger
import zipfile
import os
from textClassifier.utils.commons import get_size
import gdown
import spacy
from nltk.corpus import stopwords
import string
import pandas as pd


class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
        self.nlp = spacy.load('en_core_web_sm')
        self.stop_words = set(stopwords.words('english'))


    def text_process(self, text: str) -> str:
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]
        return ' '.join(tokens)
    
    def get_data_from_features(self) -> pd.DataFrame:
        data = pd.read_csv(self.config.training_data_file)
        data['cleaned_text'] = data['text'].apply(self.text_process)
        return data
    
    def save_data(self, data: pd.DataFrame, path_name: str='cleaned_tweets.csv') -> None:
        out_path_name = Path(self.config.training_cleansed_data)
        data.to_csv(out_path_name, index=False)

        





        

NameError: name 'DataPreprocessingConfig' is not defined

In [ ]:
from textClassifier import logger
import pandas as pd
import spacy
from nltk.corpus import stopwords
import string
from pathlib import Path
import torch
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
        self.nlp = spacy.load('en_core_web_sm')  # Load spaCy model
        self.stop_words = set(stopwords.words('english'))  # Load stopwords
        self.mapping_labels = {'negative': 0, 'neutral': 1, 'positive': 2}
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        

    def text_process(self, text: str) -> str:
        """
        Preprocesses the input text by:
        1. Converting to lowercase
        2. Removing punctuation
        3. Tokenizing and lemmatizing
        4. Removing stopwords
        """
        # Convert to lowercase
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize and lemmatize
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]

        return ' '.join(tokens)

    def get_data_from_features(self) -> pd.DataFrame:
        """
        Loads the dataset from the path specified in the config and applies preprocessing to the text column.
        """
        # Check if the file exists
        if not Path(self.config.training_data_file).exists():
            raise FileNotFoundError(f"File not found: {self.config.training_data_file}")

        # Load the dataset
        data = pd.read_csv(self.config.training_data_file)

        # Apply preprocessing to the text column
        data['cleaned_text'] = data['text'].apply(self.text_process)

        return data

    def save_data(self, data: pd.DataFrame, path_name: str = 'cleaned_tweets.csv') -> None:
        """
        Saves the preprocessed data to the specified output file.

        Args:
            data (pd.DataFrame): The preprocessed DataFrame to save.
            path_name (str): The name of the output file. Defaults to "cleaned_tweets.csv".
        """
        # Create the full output file path
        out_path_name = Path(self.config.training_cleansed_data) / path_name

        # Save the DataFrame to CSV
        data.to_csv(out_path_name, index=False)
        logger.info(f"Preprocessed data saved to {out_path_name}")


    def mapping_labels_func(self, data: pd.DataFrame) -> pd.DataFrame:

        data['label'] = data['airline_sentiment'].map(self.mapping_labels)
        logger.info(f"values are mapped in the {data}")
        return data
    
    # def tokenizing_func(self, data: pd.DataFrame, text_column: str = 'cleaned_text') -> dict:
    #     # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    #     # return tokenizer(data, padding='max_length', truncation=True, max_length=128)
    
    #     tokenized_inputs = self.tokenizer(
    #         data[text_column].tolist(),  # Convert text column to list
    #         padding='max_length',        # Pad to max length
    #         truncation=True,             # Truncate to max length
    #         max_length=128,              # Set max length
    #         return_tensors='pt'          # Return PyTorch tensors
    #     )
    #     return tokenized_inputs

    def tokenize_text(self, data: pd.DataFrame, text_column: str = 'cleaned_text') -> pd.DataFrame:
        """
        Tokenizes the text in the specified column using BERT tokenizer.

        Args:
            data (pd.DataFrame): The DataFrame containing the text to tokenize.
            text_column (str): The name of the column containing the text. Defaults to 'cleaned_text'.

        Returns:
            pd.DataFrame: The DataFrame with an additional 'tokenized' column.
        """
        # Tokenize the text
        data['tokenized'] = data[text_column].apply(
            lambda x: self.tokenizer(x, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        )
        logger.info(f"Text in column '{text_column}' has been tokenized.")
        return data
    
    def train_val_test_split(self, data: pd.DataFrame, test_size: float = 0.3, val_size: float = 0.5, random_state: int = 42):
        """
        Splits the dataset into training, validation, and test sets.

        Args:
            data (pd.DataFrame): The DataFrame containing the tokenized data and labels.
            test_size (float): Proportion of the dataset to include in the test split. Defaults to 0.3.
            val_size (float): Proportion of the temp split to include in the validation split. Defaults to 0.5.
            random_state (int): Random seed for reproducibility. Defaults to 42.

        Returns:
            A dictionary containing the splits:
            {
                'train': {'texts': train_texts, 'labels': train_labels},
                'val': {'texts': val_texts, 'labels': val_labels},
                'test': {'texts': test_texts, 'labels': test_labels}
            }
        """
        # Split into train and temp (val + test)
        train_texts, temp_texts, train_labels, temp_labels = train_test_split(
            data['tokenized'].to_list(), data['label'].to_list(), test_size=test_size, random_state=random_state
        )

        # Split temp into validation and test
        val_texts, test_texts, val_labels, test_labels = train_test_split(
            temp_texts, temp_labels, test_size=val_size, random_state=random_state
        )

        logger.info(f"Data split into train, validation, and test sets.")
        return {
            'train': {'texts': train_texts, 'labels': train_labels},
            'val': {'texts': val_texts, 'labels': val_labels},
            'test': {'texts': test_texts, 'labels': test_labels}
        }




c:\Users\prass\miniconda3\envs\tcenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
try:
    config = ConfigurationManager()
    get_data_pre_config = config.get_data_preprocessing_config()
    data_pre_process = DataPreprocessing(config=get_data_pre_config)
    data_pre_process.text_process()
    output = data_pre_process.get_data_from_features()
    data_pre_process.save_data(data=output, path_name='cleaned_tweets.csv')
    output = data_pre_process.mapping_labels_func(data=output)
    output = data_pre_process.tokenize_text(data=output)
    

    
except Exception as e:
    raise e

[2025-03-12 12:14:12,361: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-03-12 12:14:12,381: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-03-12 12:14:12,384: INFO: commons: created directory at: artifacts]
[2025-03-12 12:14:12,386: INFO: commons: created directory at: artifacts/feature_engineering]


TypeError: text_process() missing 1 required positional argument: 'text'

In [ ]:
# Example usage
if __name__ == "__main__":
    # Assuming ConfigurationManager is defined elsewhere
    config_manager = ConfigurationManager()

    # Get the data preprocessing config
    get_data_pre_config = config_manager.get_data_preprocessing_config()

    # Initialize DataPreprocessing with the config
    data_processing = DataPreprocessing(config=get_data_pre_config)

    # Load and preprocess the data
    output = data_processing.get_data_from_features()

    # Save the preprocessed data
    data_processing.save_data(data=output, path_name='cleaned_tweets.csv')

    # Map labels to numerical values
    output = data_processing.mapping_labels_func(data=output)

    # Tokenize the cleaned text
    output = data_processing.tokenize_text(data=output)

    # Split the data into train, validation, and test sets
    splits = data_processing.train_val_test_split(data=output)

    # Access the splits
    train_data = splits['train']
    val_data = splits['val']
    test_data = splits['test']

    # Display the sizes of the splits
    print(f"Train set size: {len(train_data['texts'])}")
    print(f"Validation set size: {len(val_data['texts'])}")
    print(f"Test set size: {len(test_data['texts'])}")

[2025-02-14 15:32:52,577: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-14 15:32:52,590: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-14 15:32:52,593: INFO: commons: created directory at: artifacts]
[2025-02-14 15:32:52,600: INFO: commons: created directory at: artifacts/feature_engineering]
[2025-02-14 15:34:45,125: INFO: 2396406771: Preprocessed data saved to artifacts\feature_engineering\cleaned_tweets.csv]
[2025-02-14 15:34:45,136: INFO: 2396406771: values are mapped in the                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837 

In [51]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

def convert_to_dicts(tokenized_texts):
    """
    Converts a list of tokenized texts into a dictionary of input IDs and attention masks.

    Args:
        tokenized_texts (list): List of tokenized texts (each is a dictionary with 'input_ids' and 'attention_mask').

    Returns:
        dict: A dictionary containing 'input_ids' and 'attention_mask' as lists.
    """
    input_ids = [d['input_ids'].squeeze(0) for d in tokenized_texts]  # Remove batch dimension
    attention_masks = [d['attention_mask'].squeeze(0) for d in tokenized_texts]  # Remove batch dimension
    return {'input_ids': input_ids, 'attention_mask': attention_masks}


def create_datasets(train_texts, train_labels, val_texts, val_labels, test_texts, test_labels):
    """
    Creates SentimentDataset objects for train, validation, and test splits.

    Args:
        train_texts (list): List of tokenized texts for the training set.
        train_labels (list): List of labels for the training set.
        val_texts (list): List of tokenized texts for the validation set.
        val_labels (list): List of labels for the validation set.
        test_texts (list): List of tokenized texts for the test set.
        test_labels (list): List of labels for the test set.

    Returns:
        train_dataset, val_dataset, test_dataset: SentimentDataset objects for each split.
    """
    # Convert tokenized texts to encodings
    train_encodings = convert_to_dicts(train_texts)
    val_encodings = convert_to_dicts(val_texts)
    test_encodings = convert_to_dicts(test_texts)

    # Create SentimentDataset objects
    train_dataset = SentimentDataset(train_encodings, train_labels)
    val_dataset = SentimentDataset(val_encodings, val_labels)
    test_dataset = SentimentDataset(test_encodings, test_labels)

    return train_dataset, val_dataset, test_dataset

In [52]:
# Example usage
if __name__ == "__main__":
    # Assuming ConfigurationManager is defined elsewhere
    config_manager = ConfigurationManager()

    # Get the data preprocessing config
    get_data_pre_config = config_manager.get_data_preprocessing_config()

    # Initialize DataPreprocessing with the config
    data_processing = DataPreprocessing(config=get_data_pre_config)

    # Load and preprocess the data
    output = data_processing.get_data_from_features()

    # Save the preprocessed data
    data_processing.save_data(data=output, path_name='cleaned_tweets.csv')

    # Map labels to numerical values
    output = data_processing.mapping_labels_func(data=output)

    # Tokenize the cleaned text
    output = data_processing.tokenize_text(data=output)

    # Split the data into train, validation, and test sets
    splits = data_processing.train_val_test_split(data=output)

    # Access the splits
    train_data = splits['train']
    val_data = splits['val']
    test_data = splits['test']

    # Create datasets
    train_dataset, val_dataset, test_dataset = create_datasets(
        train_data['texts'], train_data['labels'],
        val_data['texts'], val_data['labels'],
        test_data['texts'], test_data['labels']
    )

    # Example of accessing data
    print(train_dataset[0])

[2025-02-14 15:50:20,078: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-14 15:50:20,082: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-14 15:50:20,084: INFO: commons: created directory at: artifacts]
[2025-02-14 15:50:20,086: INFO: commons: created directory at: artifacts/feature_engineering]
[2025-02-14 15:52:16,736: INFO: 2396406771: Preprocessed data saved to artifacts\feature_engineering\cleaned_tweets.csv]
[2025-02-14 15:52:16,745: INFO: 2396406771: values are mapped in the                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837 

C:\Users\prass\AppData\Local\Temp\ipykernel_17580\2803380171.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# Map labels to integers
# label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
# data['label'] = data['airline_sentiment'].map(label_mapping)

# from transformers import BertTokenizer

# # Tokenization
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize the data
# def tokenize_function(text):
#     return tokenizer(text, padding='max_length', truncation=True, max_length=128)

# data['tokenized'] = data['cleaned_text'].apply(tokenize_function)

In [30]:
config_manager = ConfigurationManager()

# Get the data preprocessing config
get_data_pre_config = config_manager.get_data_preprocessing_config()

# Initialize DataPreprocessing with the config
data_processing = DataPreprocessing(config=get_data_pre_config)

# Load and preprocess the data
output = data_processing.get_data_from_features()

# Save the preprocessed data
data_processing.save_data(data=output, path_name='cleaned_tweets.csv')

# mapping = data_processing.mapping_labels_func()
output = data_processing.mapping_labels_func(data = output)
print(output.head())
tokenized_data = data_processing.tokenizing_func(data=output)
print(tokenized_data[:5])



[2025-02-12 15:56:11,364: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-12 15:56:11,368: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-12 15:56:11,370: INFO: commons: created directory at: artifacts]
[2025-02-12 15:56:11,373: INFO: commons: created directory at: artifacts/feature_engineering]
[2025-02-12 15:58:20,522: INFO: 751830446: Preprocessed data saved to artifacts\feature_engineering\cleaned_tweets.csv]
[2025-02-12 15:58:20,533: INFO: 751830446: values are mapped in the                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   

In [47]:
@staticmethod
def train_test_split_func(tokenized_data, labels, test_size=0.3, val_size=0.5, random_state=42):
    # Split into train and temp (val + test)
    train_texts, temp_texts, train_labels, temp_labels = train_test_split(
        tokenized_data['input_ids'], labels, test_size=test_size, random_state=random_state
        )
        
        # Split temp into validation and test
    val_texts, test_texts, val_labels, test_labels = train_test_split(
            temp_texts, temp_labels, test_size=val_size, random_state=random_state
        )
        
    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels

In [ ]:
# from datasets import Dataset
# from transformers import AutoTokenizer

# class DataPreprocessing:
#     def __init__(self, config):
#         """
#         Initializes the DataPreprocessing class.

#         Args:
#             config: Configuration object containing paths and settings.
#         """
#         self.config = config
#         self.nlp = spacy.load('en_core_web_sm')  # Load spaCy model
#         self.stop_words = set(stopwords.words('english'))  # Load stopwords
#         self.mapping_labels = {'negative': 0, 'neutral': 1, 'positive': 2}
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        
#         # Load the dataset during initialization
#         self.data = self._load_data()

#         # Set the default output file path
#         self.output_file_path = Path(self.config.training_cleansed_data) / 'cleaned_tweets.csv'

#     def _load_data(self) -> pd.DataFrame:
#         """
#         Loads the dataset from the path specified in the config.

#         Returns:
#             pd.DataFrame: The loaded dataset.
#         """
#         # Check if the file exists
#         if not Path(self.config.training_data_file).exists():
#             raise FileNotFoundError(f"File not found: {self.config.training_data_file}")

#         # Load the dataset
#         data = pd.read_csv(self.config.training_data_file)
#         logger.info(f"Dataset loaded from {self.config.training_data_file}")
#         return data

#     def text_process(self) -> None:
#         """
#         Preprocesses the text column in the dataset by:
#         1. Converting to lowercase
#         2. Removing punctuation
#         3. Tokenizing and lemmatizing
#         4. Removing stopwords
#         """
#         # Apply preprocessing to the text column
#         self.data['cleaned_text'] = self.data['text'].apply(self._process_single_text)
#         logger.info("Text preprocessing completed.")

#     def _process_single_text(self, text: str) -> str:
#         """
#         Helper method to preprocess a single text string.

#         Args:
#             text (str): The input text to preprocess.

#         Returns:
#             str: The preprocessed text.
#         """
#         # Convert to lowercase
#         text = text.lower()

#         # Remove punctuation
#         text = text.translate(str.maketrans('', '', string.punctuation))

#         # Tokenize and lemmatize
#         doc = self.nlp(text)
#         tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]

#         return ' '.join(tokens)

#     def mapping_labels_func(self) -> None:
#         """
#         Maps the sentiment labels to numerical values.
#         """
#         self.data['label'] = self.data['airline_sentiment'].map(self.mapping_labels)
#         logger.info("Labels mapped to numerical values.")

#     def tokenize_text(self) -> None:
#         """
#         Tokenizes the cleaned text column using BERT tokenizer.
#         """
#         # Tokenize the text
#         self.data['tokenized'] = self.data['cleaned_text'].apply(
#             lambda x: self.tokenizer(x, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
#         )
#         logger.info("Text tokenization completed.")

#     def save_data(self) -> None:
#         """
#         Saves the preprocessed data to the default output file.
#         """
#         # Save the DataFrame to CSV
#         self.data.to_csv(self.output_file_path, index=False)
#         logger.info(f"Preprocessed data saved to {self.output_file_path}")

#     def train_val_test_split(self, test_size: float = 0.3, val_size: float = 0.5, random_state: int = 42) -> dict:
#         """
#         Splits the dataset into training, validation, and test sets.

#         Args:
#             test_size (float): Proportion of the dataset to include in the test split. Defaults to 0.3.
#             val_size (float): Proportion of the temp split to include in the validation split. Defaults to 0.5.
#             random_state (int): Random seed for reproducibility. Defaults to 42.

#         Returns:
#             A dictionary containing the splits:
#             {
#                 'train': {'texts': train_texts, 'labels': train_labels},
#                 'val': {'texts': val_texts, 'labels': val_labels},
#                 'test': {'texts': test_texts, 'labels': test_labels}
#             }
#         """
#         # Split into train and temp (val + test)
#         train_texts, temp_texts, train_labels, temp_labels = train_test_split(
#             self.data['cleaned_text'].to_list(), self.data['label'].to_list(), test_size=test_size, random_state=random_state
#         )

#         # Split temp into validation and test
#         val_texts, test_texts, val_labels, test_labels = train_test_split(
#             temp_texts, temp_labels, test_size=val_size, random_state=random_state
#         )

#         logger.info("Data split into train, validation, and test sets.")
#         return {
#             'train': {'texts': train_texts, 'labels': train_labels},
#             'val': {'texts': val_texts, 'labels': val_labels},
#             'test': {'texts': test_texts, 'labels': test_labels}
#         }

#     def convert_to_tokenized_datasets(self, splits: dict) -> dict:
#         """
#         Converts the train, validation, and test splits into tokenized datasets.

#         Args:
#             splits (dict): A dictionary containing the splits:
#                 {
#                     'train': {'texts': train_texts, 'labels': train_labels},
#                     'val': {'texts': val_texts, 'labels': val_labels},
#                     'test': {'texts': test_texts, 'labels': test_labels}
#                 }

#         Returns:
#             dict: A dictionary containing the tokenized datasets:
#                 {
#                     'train': tokenized_train_dataset,
#                     'val': tokenized_val_dataset,
#                     'test': tokenized_test_dataset
#                 }
#         """
#         # Convert splits to DataFrames
#         train_df = pd.DataFrame({'cleaned_text': splits['train']['texts'], 'label': splits['train']['labels']})
#         val_df = pd.DataFrame({'cleaned_text': splits['val']['texts'], 'label': splits['val']['labels']})
#         test_df = pd.DataFrame({'cleaned_text': splits['test']['texts'], 'label': splits['test']['labels']})

#         # Convert DataFrames to datasets.Dataset
#         train_dataset = Dataset.from_pandas(train_df)
#         val_dataset = Dataset.from_pandas(val_df)
#         test_dataset = Dataset.from_pandas(test_df)

#         # Tokenize the datasets
#         def tokenize_data(examples):
#             return self.tokenizer(examples["cleaned_text"], truncation=True, padding='max_length', max_length=128)

#         tokenized_train = train_dataset.map(tokenize_data, batched=True)
#         tokenized_val = val_dataset.map(tokenize_data, batched=True)
#         tokenized_test = test_dataset.map(tokenize_data, batched=True)

#         logger.info("Datasets tokenized successfully.")
#         return {
#             'train': tokenized_train,
#             'val': tokenized_val,
#             'test': tokenized_test
#         }

In [ ]:
try:
    config = ConfigurationManager()
    get_data_pre_config = config.get_data_preprocessing_config()
    data_pre_process = DataPreprocessing(config=get_data_pre_config)
    
    # Preprocess the data
    data_pre_process.text_process()
    data_pre_process.mapping_labels_func()
    data_pre_process.tokenize_text()
    
    # Save the preprocessed data
    data_pre_process.save_data()  # No need to pass the path
    
    # Split the data into train, validation, and test sets
    splits = data_pre_process.train_val_test_split()
    
    # Convert splits to tokenized datasets
    tokenized_datasets = data_pre_process.convert_to_tokenized_datasets(splits)
    
    # Access the tokenized datasets
    train_dataset = tokenized_datasets['train']
    val_dataset = tokenized_datasets['val']
    test_dataset = tokenized_datasets['test']
    
    logger.info("Tokenized datasets created successfully.")
    
except Exception as e:
    logger.error(f"Error during data preprocessing: {e}")
    raise e

In [23]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    training_data_path: Path
    training_data_file: Path
    training_cleansed_data: Path
    datasets_dir: Path


In [25]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.feature_engineering

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            training_data_path=config.training_data_path,
            training_data_file=config.training_data_file,
            training_cleansed_data=config.training_cleansed_data,
            datasets_dir=config.datasets_dir
        )

        return data_preprocessing_config

In [26]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import spacy
from nltk.corpus import stopwords
import string
import logging

logger = logging.getLogger(__name__)

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        """
        Custom PyTorch Dataset for sentiment analysis.

        Args:
            encodings (dict): Tokenized encodings (e.g., input_ids, attention_mask).
            labels (list): List of labels corresponding to the encodings.
        """
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

class DataPreprocessing:
    def __init__(self, config):
        """
        Initializes the DataPreprocessing class.

        Args:
            config: Configuration object containing paths and settings.
        """
        self.config = config
        self.nlp = spacy.load('en_core_web_sm')  # Load spaCy model
        self.stop_words = set(stopwords.words('english'))  # Load stopwords
        self.mapping_labels = {'negative': 0, 'neutral': 1, 'positive': 2}
        # self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
        
        # Load the dataset during initialization
        self.data = self._load_data()

        # Set the default output file path
        self.output_file_path = Path(self.config.training_cleansed_data) / 'cleaned_tweets.csv'
        self.datasets_dir = Path(self.config.datasets_dir)

    def _load_data(self) -> pd.DataFrame:
        """
        Loads the dataset from the path specified in the config.

        Returns:
            pd.DataFrame: The loaded dataset.
        """
        # Check if the file exists
        if not Path(self.config.training_data_file).exists():
            raise FileNotFoundError(f"File not found: {self.config.training_data_file}")

        # Load the dataset
        data = pd.read_csv(self.config.training_data_file)
        logger.info(f"Dataset loaded from {self.config.training_data_file}")
        return data

    def text_process(self) -> None:
        """
        Preprocesses the text column in the dataset by:
        1. Converting to lowercase
        2. Removing punctuation
        3. Tokenizing and lemmatizing
        4. Removing stopwords
        """
        # Apply preprocessing to the text column
        self.data['cleaned_text'] = self.data['text'].apply(self._process_single_text)
        logger.info("Text preprocessing completed.")

    def _process_single_text(self, text: str) -> str:
        """
        Helper method to preprocess a single text string.

        Args:
            text (str): The input text to preprocess.

        Returns:
            str: The preprocessed text.
        """
        # Convert to lowercase
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize and lemmatize
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]

        return ' '.join(tokens)

    def mapping_labels_func(self) -> None:
        """
        Maps the sentiment labels to numerical values.
        """
        self.data['label'] = self.data['airline_sentiment'].map(self.mapping_labels)
        logger.info("Labels mapped to numerical values.")

    def tokenize_text(self) -> None:
        """
        Tokenizes the cleaned text column using BERT tokenizer.
        """
        # Tokenize the text
        self.data['tokenized'] = self.data['cleaned_text'].apply(
            lambda x: self.tokenizer(x, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        )
        logger.info("Text tokenization completed.")

    def save_data(self) -> None:
        """
        Saves the preprocessed data to the default output file.
        """
        # Save the DataFrame to CSV
        self.data.to_csv(self.output_file_path, index=False)
        logger.info(f"Preprocessed data saved to {self.output_file_path}")

    def train_val_test_split(self, test_size: float = 0.3, val_size: float = 0.5, random_state: int = 42) -> dict:
        """
        Splits the dataset into training, validation, and test sets.

        Args:
            test_size (float): Proportion of the dataset to include in the test split. Defaults to 0.3.
            val_size (float): Proportion of the temp split to include in the validation split. Defaults to 0.5.
            random_state (int): Random seed for reproducibility. Defaults to 42.

        Returns:
            A dictionary containing the splits:
            {
                'train': {'texts': train_texts, 'labels': train_labels},
                'val': {'texts': val_texts, 'labels': val_labels},
                'test': {'texts': test_texts, 'labels': test_labels}
            }
        """
        # Split into train and temp (val + test)
        train_texts, temp_texts, train_labels, temp_labels = train_test_split(
            self.data['cleaned_text'].to_list(), self.data['label'].to_list(), test_size=test_size, random_state=random_state
        )

        # Split temp into validation and test
        val_texts, test_texts, val_labels, test_labels = train_test_split(
            temp_texts, temp_labels, test_size=val_size, random_state=random_state
        )

        logger.info("Data split into train, validation, and test sets.")
        return {
            'train': {'texts': train_texts, 'labels': train_labels},
            'val': {'texts': val_texts, 'labels': val_labels},
            'test': {'texts': test_texts, 'labels': test_labels}
        }

    def convert_to_tokenized_datasets(self, splits: dict) -> dict:
        """
        Converts the train, validation, and test splits into PyTorch datasets.

        Args:
            splits (dict): A dictionary containing the splits:
                {
                    'train': {'texts': train_texts, 'labels': train_labels},
                    'val': {'texts': val_texts, 'labels': val_labels},
                    'test': {'texts': test_texts, 'labels': test_labels}
                }

        Returns:
            dict: A dictionary containing the PyTorch datasets:
                {
                    'train': train_dataset,
                    'val': val_dataset,
                    'test': test_dataset
                }
        """
        # Tokenize the texts
        train_encodings = self.tokenizer(splits['train']['texts'], truncation=True, padding=True, max_length=128)
        val_encodings = self.tokenizer(splits['val']['texts'], truncation=True, padding=True, max_length=128)
        test_encodings = self.tokenizer(splits['test']['texts'], truncation=True, padding=True, max_length=128)

        # Create PyTorch datasets
        train_dataset = SentimentDataset(train_encodings, splits['train']['labels'])
        val_dataset = SentimentDataset(val_encodings, splits['val']['labels'])
        test_dataset = SentimentDataset(test_encodings, splits['test']['labels'])

        logger.info("PyTorch datasets created successfully.")
        return {
            'train': train_dataset,
            'val': val_dataset,
            'test': test_dataset
        }
    def save_datasets(self, train_dataset, val_dataset, test_dataset):
        """
        Saves the train, validation, and test datasets to the specified directory.

        Args:
            train_dataset: Training dataset.
            val_dataset: Validation dataset.
            test_dataset: Test dataset.
        """
        # Create the directory if it doesn't exist
        datasets_dir = Path(self.config.datasets_dir)
        datasets_dir.mkdir(parents=True, exist_ok=True)

        # Save the datasets
        torch.save(train_dataset, datasets_dir / "train_dataset.pt")
        torch.save(val_dataset, datasets_dir / "val_dataset.pt")
        torch.save(test_dataset, datasets_dir / "test_dataset.pt")

        logger.info(f"Datasets saved to {datasets_dir}")

In [27]:
try:
    config = ConfigurationManager()
    get_data_pre_config = config.get_data_preprocessing_config()
    data_pre_process = DataPreprocessing(config=get_data_pre_config)
    
    # Preprocess the data
    data_pre_process.text_process()
    data_pre_process.mapping_labels_func()
    data_pre_process.tokenize_text()
    
    # Save the preprocessed data
    data_pre_process.save_data()  # No need to pass the path
    
    # Split the data into train, validation, and test sets
    splits = data_pre_process.train_val_test_split()
    
    # Convert splits to PyTorch datasets
    tokenized_datasets = data_pre_process.convert_to_tokenized_datasets(splits)
    
    # Access the PyTorch datasets
    train_dataset = tokenized_datasets['train']
    val_dataset = tokenized_datasets['val']
    test_dataset = tokenized_datasets['test']
    logger.info("PyTorch datasets created successfully.")
    datasets = data_pre_process.save_datasets(train_dataset, val_dataset, test_dataset)
    
except Exception as e:
    logger.error(f"Error during data preprocessing: {e}")
    raise e

[2025-03-13 12:07:14,560: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-03-13 12:07:14,573: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-03-13 12:07:14,580: INFO: commons: created directory at: artifacts]
[2025-03-13 12:07:14,594: INFO: commons: created directory at: artifacts/feature_engineering]
[2025-03-13 12:07:16,666: INFO: 2686220794: Dataset loaded from artifacts/feature_engineering/Tweets.csv]
[2025-03-13 12:09:10,854: INFO: 2686220794: Text preprocessing completed.]
[2025-03-13 12:09:10,873: INFO: 2686220794: Labels mapped to numerical values.]
[2025-03-13 12:09:15,663: INFO: 2686220794: Text tokenization completed.]
[2025-03-13 12:09:31,817: INFO: 2686220794: Preprocessed data saved to artifacts\feature_engineering\cleaned_tweets.csv]
[2025-03-13 12:09:31,834: INFO: 2686220